# Storing API Requests in MongoDB
What we want to do is to store the requests in such a way that we can make use of the objects and classes to analyse the league and results.  

This builds on the other notebooks that have written that show the basics of API calls and MongoDB. 

In [1]:
# Import packages 
import json
import os

import pandas as pd
import pymongo

import requests

In [2]:
# Get the database details from the conda environment 
mongodb_password = os.environ.get("mongodb_password")
mongodb_user = os.environ.get("mongodb_user")

When we end up automating this, we are going to want to set a hard limit and control the requests, throttling our access to make sure we don't incure any additional costs for the tier. 

In [3]:
# Define max requests for this run 
api_max_requests = 10

In [4]:
# Access the database client and define database 
client = pymongo.MongoClient("mongodb+srv://" + mongodb_user + ":" + mongodb_password + "@basiccluster-6s0er.mongodb.net/test?retryWrites=true&w=majority")
db = client.football

In [5]:
# Get the API details from the conda environment 
headers = {
    'x-rapidapi-host': os.environ.get("api_host"), 
    'x-rapidapi-key': os.environ.get("api_key") 
    }

def api_call(url):
    response = requests.request("GET", url, headers=headers)
    
    # Return a dictionary of the response text
    return json.loads(response.text)

For now, I'm going to focus on the premier league. If required, we can extend and restructure.

In [23]:
# Define the base URL for the API requests
base_url = 'https://api-football-v1.p.rapidapi.com/v2/'
premier_league_id = '524'
fixtures_url = base_url + 'fixtures/league/' + premier_league_id

# Call the league fixture list and compare to current version
fixtures = api_call(fixtures_url)

In [24]:
# Grab the particular fixture information, and remove the unnecessary response data
fixtures = fixtures['api']['fixtures']

In [21]:
db.fixtures.insert_many(fixtures)

/Users/willcanniford/anaconda3/envs/environment/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


[ObjectId('5dc48549808a6c58861183b1'),
 ObjectId('5dc48549808a6c58861183b2'),
 ObjectId('5dc48549808a6c58861183b3'),
 ObjectId('5dc48549808a6c58861183b4'),
 ObjectId('5dc48549808a6c58861183b5'),
 ObjectId('5dc48549808a6c58861183b6'),
 ObjectId('5dc48549808a6c58861183b7'),
 ObjectId('5dc48549808a6c58861183b8'),
 ObjectId('5dc48549808a6c58861183b9'),
 ObjectId('5dc48549808a6c58861183ba'),
 ObjectId('5dc48549808a6c58861183bb'),
 ObjectId('5dc48549808a6c58861183bc'),
 ObjectId('5dc48549808a6c58861183bd'),
 ObjectId('5dc48549808a6c58861183be'),
 ObjectId('5dc48549808a6c58861183bf'),
 ObjectId('5dc48549808a6c58861183c0'),
 ObjectId('5dc48549808a6c58861183c1'),
 ObjectId('5dc48549808a6c58861183c2'),
 ObjectId('5dc48549808a6c58861183c3'),
 ObjectId('5dc48549808a6c58861183c4'),
 ObjectId('5dc48549808a6c58861183c5'),
 ObjectId('5dc48549808a6c58861183c6'),
 ObjectId('5dc48549808a6c58861183c7'),
 ObjectId('5dc48549808a6c58861183c8'),
 ObjectId('5dc48549808a6c58861183c9'),
 ObjectId('5dc48549808a6c

## Filtering the fixtures

We have inserted the 380 fixtures for the premier league for this season. Going forward we are going to want to call this again and then just update those that have changed i.e. games that have been played. 

In [53]:
completed_matches = list(db.fixtures.find({'status': 'Match Finished'}))

In [54]:
len(completed_matches)

110

There have been 110 completed_matches so far this season according to our records.  

These are the matches that we are going to want to further get the stats for and store against the fixture. 

In [80]:
postponed_matches = list(db.fixtures.find(
    {"$and": [
        {"status": {
            "$ne": 'Match Finished'
        }},
        {"status": {
            "$ne": 'Not Started'
        }}
    ]}
))

In [81]:
postponed_matches

[{'_id': ObjectId('5dc48549808a6c588611845b'),
  'fixture_id': 157194,
  'league_id': 524,
  'event_date': '2019-12-20T00:00:00+00:00',
  'event_timestamp': 1576800000,
  'firstHalfStart': None,
  'secondHalfStart': None,
  'round': 'Regular Season - 18',
  'status': 'Match Postponed',
  'statusShort': 'PST',
  'elapsed': 0,
  'venue': 'London Stadium (London)',
  'referee': None,
  'homeTeam': {'team_id': 48,
   'team_name': 'West Ham',
   'logo': 'https://media.api-football.com/teams/48.png'},
  'awayTeam': {'team_id': 40,
   'team_name': 'Liverpool',
   'logo': 'https://media.api-football.com/teams/40.png'},
  'goalsHomeTeam': None,
  'goalsAwayTeam': None,
  'score': {'halftime': None,
   'fulltime': None,
   'extratime': None,
   'penalty': None}}]

In [82]:
incomplete_matches = list(db.fixtures.find({'status': {
            "$ne": 'Match Finished'
        }}, {'fixture_id'}))

Get the fixture IDs for those matches that haven't been completed yet. 

In [83]:
[match['fixture_id'] for match in incomplete_matches]

[157131,
 157126,
 157125,
 157130,
 157132,
 157133,
 157127,
 157129,
 157134,
 157128,
 157144,
 157135,
 157136,
 157138,
 157139,
 157140,
 157143,
 157141,
 157142,
 157137,
 157150,
 157145,
 157146,
 157148,
 157153,
 157152,
 157151,
 157154,
 157147,
 157149,
 157163,
 157156,
 157157,
 157159,
 157160,
 157161,
 157162,
 157164,
 157158,
 157155,
 157168,
 157165,
 157172,
 157173,
 157169,
 157166,
 157170,
 157171,
 157167,
 157174,
 157180,
 157176,
 157177,
 157179,
 157182,
 157183,
 157181,
 157184,
 157175,
 157178,
 157194,
 157188,
 157185,
 157186,
 157187,
 157190,
 157191,
 157189,
 157193,
 157192,
 157203,
 157195,
 157196,
 157197,
 157198,
 157199,
 157202,
 157201,
 157200,
 157204,
 157206,
 157210,
 157212,
 157213,
 157211,
 157214,
 157207,
 157205,
 157208,
 157209,
 157216,
 157217,
 157220,
 157222,
 157223,
 157219,
 157221,
 157224,
 157215,
 157218,
 157232,
 157228,
 157227,
 157229,
 157230,
 157231,
 157234,
 157233,
 157225,
 157226,
 157243,
 